1. http://mlg.eng.cam.ac.uk/yarin/blog_3d801aa532c1ce.html
2. "Uncertainty in deep learning" Yarin Gal

In [ ]:
%matplotlib tk
%load_ext autoreload
%autoreload 2


In [ ]:
import numpy as np
from ..bayesian.callbacks import ModelTest
from keras.optimizers import SGD, RMSprop, Adagrad
from keras.models import Sequential, Model
from keras.models import Input
from keras.layers.core import Dense, Dropout
from keras.layers import Concatenate
from keras.layers.embeddings import Embedding
from keras.layers.recurrent import LSTM, GRU, SimpleRNN
from keras.regularizers import l2
from matplotlib import pyplot as plt
from sklearn.metrics import accuracy_score
from keras.utils import to_categorical
import itertools
from keras.callbacks import Callback
from keras import backend as K
import numpy as np

In [ ]:
xy = np.array([
    [-1, -1],
    [-0.7, -0.5],
    [-0.5, -0.1],
    [-0.2, 0.5], 
    [0, 0.8],
    [0.2, 0.5],
    [0.5, 0.2],
    [0.7, 0.0],
    [1, 0.3],
    [1.2, 0.5],
    [1.5, 0.8],
    [3.5, 0.5],
    [4.0, 0.35],
    [4.5, 0.2],
    [5.0, 0.05],
    [5.5, -0.1]
])
x = xy[:, 0] *2 
y = xy[:, 1]

In [ ]:
xy = np.array([
    [-1, -1],
    [-0.7, -0.5],
    [-0.5, -0.1],
    [-0.2, 0.5], 
    [0, 0.8],
    [0.2, 0.5],
    [0.5, 0.2],
    [0.7, 0.0],
    [1, 0.3],
    [1.2, 0.5],
    [1.5, 0.8],
    [3.5, -0.5],
    [4.0, 0.35],
    [4.5, -0.2],
    [5.0, 0.05],
    [5.5, -0.5]
])
x = xy[:, 0] *2 
y = xy[:, 1]

In [ ]:
# x = np.linspace(-10, 10, 300)
# y = (1 - 2.5*np.power(x, 2)) * np.exp(-np.power(x, 2)) + np.random.normal(0, 0.05, size=len(x))

In [ ]:
def var_f(x):
    v = np.zeros(len(x))
    v[x < 2.5] = 0.1
    v[x>=2.5]= x[x>2.5]*1.4
    return v

In [ ]:
def var_f(x):
    v = np.copy(x)
    v[v<0] = 1
    v = np.log(v) * 0.08
    return v

In [ ]:
#x = np.concatenate([np.linspace(-10, -1.1, 100), np.linspace(1.1, 10, 100)])#, np.array([0, -0.1, 0.1])])
#x = np.linspace(-10, 10, 200)
x = np.random.uniform(-10, 10, 30)
def f(x):
    return (1 - 2.5*np.power(0.6*x, 2)) * np.exp(-np.power(0.6*x, 2))


# def f(x):
#     return np.exp(-np.sqrt(np.abs(x))) * np.cos(x)


y = f(x) + np.random.normal(0, 0.1, size=len(x)) * var_f(x)

In [ ]:
plt.close('all')

In [ ]:
plt.plot(x, y, 'bo')
plt.show()

In [ ]:
X_train = x.reshape(-1, 1)
y_train = y

In [ ]:
#(X_train, y_train), (X_test, y_test) = mnist.load_data()

In [ ]:
p_dropout = 0.05
weight_decay = 0.001
batch_size=500

In [ ]:
# #regression
# mean_y_train = np.mean(y_train)
# std_y_train = np.std(y_train)
# y_train = (y_train - mean_y_train) / std_y_train


In [ ]:
from bayesian import BayesianDropoutModel, bayesian_std

In [ ]:
# Build model:
print('Build model...')
model = BayesianDropoutModel()
# model.add(Dropout(p_dropout, 
#                   input_shape=(X_train.shape[1], ),
#                  ))
model.add(Dense(25,
                input_shape=(X_train.shape[1], ),
                activation='relu',
                kernel_regularizer=l2(weight_decay),
                bias_regularizer=l2(weight_decay)))
model.add(Dropout(p_dropout))
model.add(Dense(25,
                activation='sigmoid',
                kernel_regularizer=l2(weight_decay),
                bias_regularizer=l2(weight_decay)))
model.add(Dropout(p_dropout))
model.add(Dense(y_train.shape[1] if len(y_train.shape) > 1 else 1,
                activation='linear',
                kernel_regularizer=l2(weight_decay), bias_regularizer=l2(weight_decay)))

optimiser = 'adam'
model.compile(loss='mean_squared_error', optimizer=optimiser)

In [ ]:
# Build model:
print('Build model...')

# model.add(Dropout(p_dropout, 
#                   input_shape=(X_train.shape[1], ),
#                  ))
inp = Input(shape=((X_train.shape[1], )))
layer = Dense(25,
           activation='relu',
           kernel_regularizer=l2(weight_decay),
           bias_regularizer=l2(weight_decay))(inp)
layer = Dropout(p_dropout)(layer)
layer = Dense(25,
                activation='sigmoid',
                kernel_regularizer=l2(weight_decay),
                bias_regularizer=l2(weight_decay))(layer)
layer = Dropout(p_dropout)(layer)
output = Dense(y_train.shape[1] if len(y_train.shape) > 1 else 1,
                activation='linear',
                kernel_regularizer=l2(weight_decay), bias_regularizer=l2(weight_decay))(layer)

model = BayesianDropoutModel([inp], [output])

optimiser = SGD(lr=0.01, decay=1e-5)
model.compile(loss='mean_squared_error', optimizer=optimiser)

In [ ]:
for l in model.layers:
    print(type(l), l.input_shape, l.output_shape)

In [ ]:
# Potentially load weights
# model.load_weights("path")

In [ ]:
# def standardize_X(X):
#     if type(X) == list:
#         return X
#     else:
#         return [X]
    
class DrawCallback(Callback):
    def __init__(self, x, y, T, fig, ax):
        self.x = x
        self.y = y
        self.T = T
        self.ax = ax
        self.fig = fig
        #self.mean_y_train = mean_y_train
        #self.std_y_train = std_y_train
        
        #self.y = f(x)
    
    
    def on_epoch_begin(self, epoch, logs={}):
        #xx = np.linspace(x.min()-0.5, x.max()+0.5, 100).reshape(-1, 1)
        #T = 100
        prob = np.array([self.model.predict_stochastic(self.x, batch_size=500, verbose=0)
                         for _ in xrange(self.T)])
        prob_mean = np.mean(prob, 0) #* self.std_y_train + self.mean_y_train
        prob_std = bayesian_std(prob, l=1, p_dropout=p_dropout, weight_decay=weight_decay, N=len(X_train))
        
        #fig.clf()
        ax.cla()
        #ax.plot(self.x, self.y, 'b-')
        ax.plot(x, y, 'bo')
        ax.plot(self.x, prob_mean, 'r--')
        for n_std, alpha in itertools.izip([0.5, 1, 1.5], [0.3, 0.2, 0.1]):
            ax.fill_between(self.x.ravel(),
                     (prob_mean-n_std * prob_std).ravel(), 
                     (prob_mean+n_std * prob_std).ravel(), alpha=alpha, color='red')
        ax.set_xlim(self.x.min()-0.5, self.x.max()+0.5)
        ax.set_ylim(self.y.min()-0.5, self.y.max()+0.5)
        fig.canvas.draw()
        print('draw')

In [ ]:
y_train.max()

In [ ]:
plt.close('all')

In [ ]:
# Train model
print("Train...")


fig, ax = plt.subplots()
plt.show()
draw = DrawCallback(x=np.linspace(x.min()-0.5, x.max()+0.5, 100).reshape(-1, 1), 
                   y=y_train,
                   #mean_y_train=mean_y_train,
                   #std_y_train=std_y_train,
                   T=150,
                   ax = ax,
                   fig=fig)

model.fit(X_train, y_train, batch_size=batch_size, epochs=10000, 
          callbacks=[draw], verbose=1)





In [ ]:
def bayesian_mean_squared_error(y_true, pred_var):
    log_var = pred_var[:, 1]
    pred = pred_var[:, 0]
    se = K.pow(K.transpose(y_true) - pred, 2) * K.exp(-log_var) + log_var
    return K.mean(se)
    

In [ ]:
# Build model:
print('Build model...')

# model.add(Dropout(p_dropout, 
#                   input_shape=(X_train.shape[1], ),
#                  ))
inp = Input(shape=((X_train.shape[1], )))
layer = Dense(25,
           activation='relu',
           kernel_regularizer=l2(weight_decay),
           bias_regularizer=l2(weight_decay))(inp)
layer = Dropout(p_dropout)(layer)
layer = Dense(25,
                activation='relu',
                kernel_regularizer=l2(weight_decay),
                bias_regularizer=l2(weight_decay))(layer)
layer = Dropout(p_dropout)(layer)
output = Dense(1,
                activation='linear',
                kernel_regularizer=l2(weight_decay), bias_regularizer=l2(weight_decay))(layer)

var = Dense(1, activation='linear',
            kernel_regularizer=l2(weight_decay), bias_regularizer=l2(weight_decay))(layer)

final_output = Concatenate()([output, var])
model = BayesianDropoutModel([inp], [final_output])

optimiser = 'adam'
model.compile(loss=bayesian_mean_squared_error, optimizer=optimiser)

In [ ]:
# def standardize_X(X):
#     if type(X) == list:
#         return X
#     else:
#         return [X]
    
class DrawCallback2(Callback):
    def __init__(self, x, y, T, fig, ax):
        self.x = x
        self.y = y
        self.T = T
        self.ax = ax
        self.fig = fig
        #self.mean_y_train = mean_y_train
        #self.std_y_train = std_y_train
        
        #self.y = f(x)
    
    
    def on_epoch_begin(self, epoch, logs={}):
        #xx = np.linspace(x.min()-0.5, x.max()+0.5, 100).reshape(-1, 1)
        #T = 100
        pred = np.array([self.model.predict_stochastic(self.x, batch_size=500, verbose=0)
                         for _ in xrange(self.T)])
        y_pred = pred[:, :, 0]
        #print(y_pred.shape)
        prob_mean = np.mean(y_pred, 0) #* self.std_y_train + self.mean_y_train
        epistemic_prob_std = bayesian_std(y_pred, l=1, p_dropout=p_dropout, weight_decay=weight_decay, N=len(X_train))
        
        aleatoric_log_var = np.mean(pred[:, :, 1], 0)#self.model.predict(self.x)[:, 1]
        aleatoric_var = np.exp(aleatoric_log_var)
        aleatoric_std = np.sqrt(aleatoric_var)
        
        #fig.clf()
        ax.cla()
        ax.plot(self.x, f(self.x), 'b-')
        ax.plot(x, y, 'bo')
        ax.plot(self.x, prob_mean, 'r--')
        
        for n_std, alpha in itertools.izip([0.5, 1, 1.5], [0.3, 0.2, 0.1]):
            ax.fill_between(self.x.ravel(),
                     (prob_mean-n_std * epistemic_prob_std).ravel(), 
                     (prob_mean+n_std * epistemic_prob_std).ravel(), alpha=alpha, color='red')
            
            ax.fill_between(self.x.ravel(),
                     (prob_mean-n_std * aleatoric_std).ravel(), 
                     (prob_mean+n_std * aleatoric_std).ravel(), alpha=alpha, color='green')
            
        ax.set_xlim(self.x.min()-0.5, self.x.max()+0.5)
        ax.set_ylim(y_pred.min(), y_pred.max())
        ax.set_ylim(self.y.min()-0.5, self.y.max()+0.5)
        fig.canvas.draw()
        #print('draw')

In [ ]:
plt.close('all')

In [ ]:
# Train model
print("Train...")


fig, ax = plt.subplots()
plt.show()
draw = DrawCallback2(x=np.linspace(x.min()-0.5, x.max()+100.5, 200).reshape(-1, 1), 
                   y=y_train,
                   #mean_y_train=mean_y_train,
                   #std_y_train=std_y_train,
                   T=50,
                   ax = ax,
                   fig=fig)

model.fit(X_train, y_train, batch_size=batch_size, epochs=5000, 
          callbacks=[draw], verbose=1)





In [ ]:
pred = np.array([model.predict_stochastic(X_train, batch_size=500, verbose=0)
                         for _ in xrange(50)])
aleatoric_log_var = np.mean(pred[:, :, 1], 0)#self.model.predict(self.x)[:, 1]
aleatoric_var = np.exp(aleatoric_log_var)
aleatoric_std = np.sqrt(aleatoric_var)

aleatoric_std_2 = np.sqrt(np.exp(model.predict(X_train)[:, 1]))

plt.plot(x, aleatoric_std, 'ro')
plt.plot(x, aleatoric_std_2, 'bo')
plt.show()